# Method of AR6, but want forcing to act at the year end, not midyear

e.g. the solar forcing in 1750/01/01 is the action of the forcing over the 1749 calendar year. Here we get point estimates for time bounds 1750 to 2300.

Based on https://github.com/chrisroadmap/ar6 notebook 50

In [ ]:
import numpy as np
from netCDF4 import Dataset
import matplotlib.pyplot as pl
import pandas as pd
import pooch

In [ ]:
ssi14c = pooch.retrieve(
    'https://sharebox.lsce.ipsl.fr/index.php/s/LpiCUCkSmx0P6bb/download?path=%2F&files=SSI_14C_cycle_yearly_cmip_v20160613_fc.nc',
    known_hash="7a973502cdcfa492f2006bd1283827f8b0ef632efd7cf32a2d67ec350bfa2da1"
)

In [ ]:
cmip6 = pooch.retrieve(
    'http://aims3.llnl.gov/thredds/fileServer/user_pub_work/input4MIPs/CMIP6/CMIP/SOLARIS-HEPPA/SOLARIS-HEPPA-3-2/atmos/mon/multiple/gm/v20170103/solarforcing-ref-mon_input4MIPs_solar_CMIP_SOLARIS-HEPPA-3-2_gn_185001-229912.nc',
    known_hash=None
)

In [ ]:
cmip6

In [ ]:
solar_df = pd.DataFrame(index=np.arange(1750,2300))
solar_df.index.name = 'Year'

In [ ]:
nc = Dataset(ssi14c)
wl_bin = nc.variables['wavelength_bin'][:]
time   = nc.variables['time'][:]
ssi    = nc.variables['ssi'][:]
nc.close()

In [ ]:
time

In [ ]:
nc_future = Dataset(cmip6)
tsi_future = nc_future.variables['tsi'][:]
nc_future.close()

In [ ]:
len(tsi_future)

In [ ]:
years_future = np.arange(1850, 2300)
isleap = np.zeros(450)
isleap[np.logical_and(years_future%4==0, np.logical_or(years_future%100!=0, years_future%400==0))] = 1

In [ ]:
isleap[:10]

In [ ]:
pl.plot(isleap)

In [ ]:
time[8504:8605]

In [ ]:
ssi[8504]

In [ ]:
tsi = np.zeros((551))
for i, idx in enumerate(range(8504, 8605)):
    tsi[i] = np.sum(ssi[idx,:]*wl_bin)

In [ ]:
tsi[:101]

In [ ]:
for i, year in enumerate(years_future):
    weights = [31,28+isleap[i],31,30,31,30,31,31,30,31,30,31]
    tsi[i+101] = np.average(tsi_future[i*12:(i+1)*12], weights=weights)

In [ ]:
pl.plot(np.arange(1750, 2301), tsi)

In [ ]:
time[8499], np.sum(ssi[8499,:]*wl_bin)

In [ ]:
time[8500], np.sum(ssi[8500,:]*wl_bin) # start of next cycle; exclude

In [ ]:
tsi_baseline = np.sum(np.mean(ssi[1:8500,:], axis=0)*wl_bin)  # actually an error in AR6, we start from second cycle for some reason
tsi_baseline

In [ ]:
solar_erf_1750_2300 = (tsi-tsi_baseline) * 0.25 * 0.71 * 0.72

In [ ]:
df = pd.DataFrame(solar_erf_1750_2300, index=np.arange(1750,2301), columns=['erf'])
df.index.name='year'
df

In [ ]:
df.to_csv('../../data/forcing/solar_erf_timebounds.csv')